TESTING CODE-HEIGHT MAP-MUSTAFA BAL

In [1]:
##IMPORTANT LIBRARIES
"""
@author: mbal
"""

from __future__ import print_function

import os, glob, sys, threading, random, warnings, fnmatch
import shutil, random, json, h5py, re, math

import cv2
# import glob2
import numpy as np

# # matplotlib.use('agg')
import matplotlib.pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras import models
from keras import layers
from keras import backend as K
from keras.models import Model, load_model, Sequential


from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, concatenate
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

from keras.optimizers import RMSprop,Adam, SGD

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.losses import mean_squared_error
from keras.datasets import mnist
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

# from tqdm import tqdm
from itertools import chain

import scipy.io
from scipy import ndimage, misc

import sklearn
from skimage.measure import compare_psnr
from skimage.measure import compare_ssim as SSIM
from sklearn.metrics import classification_report, confusion_matrix

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label


# from google.colab import drive
from PIL import Image

from natsort import natsorted
from sklearn.utils import shuffle

os.environ["CUDA_VISIBLE_DEVICES"]="0"
print("Imported Tensorflow-Keras-OpenCV libraries")

<class 'ModuleNotFoundError'>: No module named 'h5py'

In [2]:
### NORMALIZATION
def normalize(input_data): #Normalizing the data
	return (input_data.astype(np.float32))/255

def denormalize(input_data): #Denormalizing the data
  input_data = (input_data) * 255
  return input_data.astype(np.uint8) 


In [3]:
main_dir = 'C:/Users/mbal/Desktop/mustafabal/' ## Change this directory with your current folder.
# /content/drive/My Drive/Research/HeightMAP(CC)/HEIGHTMAP DATASET/HeightMAP_Intensity Mapped
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%Y_%m_%d_%H_%M_%S")
def find_missing(lst,x):     
    return sorted(set(range(0, x )).difference(lst))

In [4]:
### LOADING THE DATA WITH THE TRAINED MODEL
def data_loader(Dataset_Folder):
  trn_file = h5py.File(main_dir+'Dataset/allnormalised224_'+Dataset_Folder+'_v2_LASTv6.h5','r')
  
  X_train = trn_file['HeightMapTrain'][:]
  Y_train= trn_file['NandSigmaTrain'][:]
  X_test = trn_file['HeightMapTest'][:]
  Y_test= trn_file['NandSigmaTest'][:]
  
  maxY_trainS=Y_train[:,1].max()  #Normalizing sigma  using global minima and maxima, 24.156723022460938
  minY_testS=Y_train[:,1].min() # 5.159702301025391
  maxY_trainN=Y_train[:,0].max()  #normalizing N 3.750767230987549
  minY_testN=Y_test[:,0].min()  #2.188645124435425
  
  print(maxY_trainS)
  print(maxY_trainN)
  print(minY_testS)
  print(minY_testN)
  
  Y_train[:,0]= Y_train[:,0]-minY_testN
  Y_train[:,0]= Y_train[:,0]/(maxY_trainN-minY_testN)
  
  Y_test[:,0]= Y_test[:,0]-minY_testN
  Y_test[:,0]= Y_test[:,0]/(maxY_trainN-minY_testN)
  
  Y_train[:,1]= Y_train[:,1]-minY_testS
  Y_train[:,1]= Y_train[:,1]/(maxY_trainS-minY_testS)
  
  Y_test[:,1]= Y_test[:,1]-minY_testS
  Y_test[:,1]= Y_test[:,1]/(maxY_trainS-minY_testS)
  
  print('X_train Size:',np.array(X_train).shape)
  print('Y_train Size:',np.array(Y_train).shape)
  print('X_test Size:',np.array(X_test).shape)
  print('Y_test Size:',np.array(Y_test).shape)
  print('X_train min-max: ',X_train.min(),X_train.max())
  print('Y_train S min-max: ',Y_train[:,1].min(),Y_train[:,1].max())
  print('Y_train N min-max: ',Y_train[:,0].min(),Y_train[:,0].max())
  print('X_test min-max: ',X_test.min(),X_test.max())
  print('Y_test N min-max: ',Y_test[:,0].min(),Y_test[:,0].max())
  print('Y_test S min-max: ',Y_test[:,1].min(),Y_test[:,1].max())
  print('-----shuffle-----')
  X_train, Y_train = shuffle(X_train, Y_train, random_state=13)
 
  print('Y_test N Variance :',np.var(Y_test[:,0]))
  print('Y_test S Variance :',np.var(Y_test[:,1]))
  
  return X_train,Y_train,X_test,Y_test

In [5]:
### TESTING RESULT- LOADING VGG-16 AND TRAINED WEIGHTS
X_train,Y_train,X_test,Y_test = data_loader('900Mhz_300m')
vgg_conv = models.load_model(main_dir+'Dataset/'+'vgg16_org.h5') ## LOADING VGG-16 MODEL
# Create the model
Mustafa_Regression = models.Sequential()
# Add the vgg convolutional base model
for layer in vgg_conv.layers[:-1]: # just exclude last layer from copying
  Mustafa_Regression.add(layer)
Mustafa_Regression.set_weights(vgg_conv.get_weights())
Mustafa_Regression.add(layers.Dense(2, activation='sigmoid'))
Mustafa_Regression.summary()

# ### CHANGE weights
Mustafa_Regression.load_weights(main_dir+'checkpoints/900Mhz_300m_All_Parameters_HEIGHT-48-0.0081.hdf5') # 


# #Predict on test
preds_test = Mustafa_Regression.predict(X_test, verbose=1)

<class 'NameError'>: name 'h5py' is not defined

In [6]:
## RESULTS
preds_N = preds_test[:,0]
preds_Sigma = preds_test[:,1]

maxY_trainS= 24.156723022460938
minY_testS= 5.159702301025391
maxY_trainN= 3.750767230987549
minY_testN=2.188645124435425

P_S_test=preds_Sigma*(maxY_trainS-minY_testS)
P_S_test=P_S_test+minY_testS
print(P_S_test.max())

P_N_test=preds_N*(maxY_trainN-minY_testN)
P_N_test=P_N_test+minY_testN
print(P_N_test.max())

Y_N_test=Y_test[:,0]*(maxY_trainN-minY_testN)
Y_N_test=Y_N_test+minY_testN
print(Y_N_test.max())

Y_S_test=Y_test[:,1]*(maxY_trainS-minY_testS)
Y_S_test=Y_S_test+minY_testS
print(Y_S_test.max())

print('Denormalized MSE N: ',np.mean((Y_N_test-P_N_test)**2))
print('Denormalized MSE Sigma: ',np.mean((Y_S_test-P_S_test)**2))

import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
fig3=plt.figure()
plt.xlim(2,4)
plt.ylim(2,4)
plt.scatter(Y_N_test, P_N_test)
plt.xlabel("True",fontsize='20')
plt.ylabel("Predicted",fontsize='20')
plt.title("True vs predicted $n$ (height maps)",fontsize='20')
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
# plt.savefig("C:/Users/mbal/Desktop/mustafabal/Wireless Communication Letter Reviews/WC_ALLPARAMETERS_HEIGHT_N_Scatter_VGG16.pdf",bbox_inches='tight')
plt.show()

import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
fig3=plt.figure()
plt.xlim(4,26)
plt.ylim(4,26)
plt.scatter(Y_S_test, P_S_test)
plt.xlabel("True",fontsize='20')
plt.ylabel("Predicted",fontsize='20')
plt.title("True vs predicted $\sigma$ (height maps)",fontsize='20')
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
# plt.savefig("C:/Users/mbal/Desktop/mustafabal/Wireless Communication Letter Reviews/WC_ALLPARAMETERS_HEIGHT_Sigma_Scatter_VGG16.pdf",bbox_inches='tight')
plt.show()

import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import matplotlib.patches as mpatches
m = np.array((np.array(Y_N_test),np.array(P_N_test)))
# print(m[0][0],m[1][0])Y_S_test=Y_train[:,1]*(maxY_trainS-minY_testS)
# Y_S_test=Y_S_test+minY_testS
print(Y_S_test.shape)
sortedArr = m [ :, m[0].argsort()]
print('Sorted 2D Numpy Array')
print(sortedArr[0][0],sortedArr[1][0])
GT=sortedArr[0]
Pred=sortedArr[1]
fig1=plt.figure()
x1 = np.linspace(0, 241, 241)
plt.plot(x1,GT,'r.',x1,Pred,'.')
plt.xlabel("Test sample numbers",fontsize='20')
plt.ylabel("$n$ values",fontsize='20')
plt.title("True vs predicted $n$ values from height maps",fontsize='20')
red_patch = mpatches.Patch(color='red', label='True $n$')
blue_patch = mpatches.Patch(color='cornflowerblue', label='Predicted $n$')
plt.legend(handles=[red_patch,blue_patch],prop={"size":18})
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.grid(True)
# plt.savefig("C:/Users/mbal/Desktop/mustafabal/Wireless Communication Letter Reviews/AllParameters_HEIGHT_N.pdf",bbox_inches='tight')


import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import matplotlib.patches as mpatches
m = np.array((np.array(Y_S_test),np.array(P_S_test)))
# print(m[0][0],m[1][0])
sortedArr = m [ :, m[0].argsort()]
print('Sorted 2D Numpy Array')
print(sortedArr[0][0],sortedArr[1][0])
GT=sortedArr[0]
Pred=sortedArr[1]
fig1=plt.figure()
x1 = np.linspace(0, 241, 241)
plt.plot(x1,GT,'r.',x1,Pred,'.')
plt.xlabel("Test sample numbers",fontsize='20')
plt.ylabel("$\sigma$ values",fontsize='20')
plt.title("True vs predicted $\sigma$ values from height maps",fontsize='20')
red_patch = mpatches.Patch(color='red', label='True $\sigma$')
blue_patch = mpatches.Patch(color='cornflowerblue', label='Predicted $\sigma$')
plt.legend(handles=[red_patch,blue_patch],prop={"size":18})
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.grid(True)
# plt.savefig("C:/Users/mbal/Desktop/mustafabal/Wireless Communication Letter Reviews/AllParameters_HEIGHT_Sigma.pdf",bbox_inches='tight')

<class 'NameError'>: name 'preds_test' is not defined